In [1]:
import pandas as pd

# Demographic Data

In [2]:
demographics_raw = pd.read_csv('raw/Student_Demographics_Raw.csv')

In [4]:
demographics_raw

,sid,school_year,male,race_ethnicity,birth_date,first_9th_school_year_reported,hs_diploma,hs_diploma_type,hs_diploma_date
0,1,2004,1,B,10869.0,2004.0,0,NaN,NaN
1,1,2005,1,H,10869.0,2004.0,0,NaN,NaN
2,1,2006,1,H,10869.0,2004.0,0,NaN,NaN
3,1,2007,1,H,10869.0,2004.0,0,NaN,NaN
4,2,2006,0,W,11948.0,NaN,1,Standard Diploma,05jun2008
...,...,...,...,...,...,...,...,...,...
87529,21803,2004,0,A,11959.0,2005.0,1,College Prep Diploma,14may2008
87530,21803,2005,0,A,11959.0,2005.0,1,College Prep Diploma,14may2008
87531,21803,2006,0,A,11959.0,2005.0,1,College Prep Diploma,14may2008
87532,21803,2007,0,A,11959.0,2005.0,1,College Prep Diploma,14may2008


In [6]:
demographics_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87534 entries, 0 to 87533
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   sid                             87534 non-null  int64  
 1   school_year                     87534 non-null  int64  
 2   male                            87534 non-null  int64  
 3   race_ethnicity                  86405 non-null  object 
 4   birth_date                      87487 non-null  float64
 5   first_9th_school_year_reported  82805 non-null  float64
 6   hs_diploma                      87534 non-null  int64  
 7   hs_diploma_type                 24701 non-null  object 
 8   hs_diploma_date                 24428 non-null  object 
dtypes: float64(2), int64(4), object(3)
memory usage: 6.0+ MB


### Let's define these features (https://hwpi.harvard.edu/files/sdp/files/step_1_stata_identify_data_specification_guide.pdf)

    - SID - a student's unique identification number
    - school year - academic school year(defined by spring semester) that the data was collected (note students may have updated attributes -- i.e. changed values  of the attributes -- each school year
    - male - whether or not a student is male or female -- 0 being female, 1 being male
    - birth_date
    - first_9th_school_year_reported - when student entered high school(accounts for students who may repeat grades; will help later in calculating length of time in hs/whether or not student met 4-year graduation rate)
    - hs_diploma - whether or not student received diploma
    - hs_diploma_type 

In [7]:
demographics_raw.shape

(87534, 9)

In [5]:
demographics_raw.describe()

,sid,school_year,male,birth_date,first_9th_school_year_reported,hs_diploma
count,87534.000000,87534.000000,87534.000000,87487.000000,82805.000000,87534.000000
mean,10909.683986,2006.471554,0.501200,12068.978031,2006.541211,0.279069
std,6296.298547,1.599801,0.500001,556.849235,1.131027,0.448544
min,1.000000,2004.000000,0.000000,10869.000000,1999.000000,0.000000
25%,5457.000000,2005.000000,0.000000,11700.000000,2006.000000,0.000000
50%,10903.500000,2006.000000,1.000000,12077.000000,2007.000000,0.000000
75%,16378.000000,2008.000000,1.000000,12447.000000,2008.000000,1.000000
max,21803.000000,2009.000000,1.000000,13148.000000,2010.000000,1.000000


In [11]:
len(demographics_raw['sid'].unique())

21803

In [19]:
demographics_raw.groupby('sid').count()

,school_year,male,race_ethnicity,birth_date,first_9th_school_year_reported,hs_diploma,hs_diploma_type,hs_diploma_date
sid,,,,,,,,
1,4,4,4,4,4,4,0,0
2,2,2,2,2,0,2,2,2
3,3,3,3,3,3,3,0,0
4,3,3,3,3,3,3,3,3
5,1,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...
21799,4,4,4,4,4,4,0,0
21800,6,6,6,6,6,6,6,6
21801,2,2,2,2,2,2,0,0


### Note that the shape of the raw data is 87534, but there are only 21803 unique student ids. The student ids should be unique identifiers. We need to clean the data so that there is only one row for each student.

But in order to do this we need to figure out student's true gender, race/ethnicity, and birthdate

In [21]:
demographics_raw.groupby('sid').groups

{1: [0, 1, 2, 3], 2: [4, 5], 3: [6, 7, 8], 4: [9, 10, 11], 5: [12], 6: [13, 14, 15], 7: [16, 17, 18, 19, 20], 8: [21, 22, 23, 24], 9: [25, 26, 27, 28], 10: [29, 30, 31, 32, 33], 11: [34], 12: [35, 36], 13: [37, 38], 14: [39, 40, 41, 42, 43, 44], 15: [45, 46], 16: [47, 48], 17: [49, 50], 18: [51, 52, 53, 54, 55], 19: [56, 57], 20: [58, 59], 21: [60, 61, 62, 63, 64, 65], 22: [66, 67, 68], 23: [69, 70, 71, 72, 73, 74], 24: [75, 76, 77, 78, 79, 80], 25: [81, 82, 83, 84, 85], 26: [86, 87, 88, 89], 27: [90, 91, 92, 93, 94, 95], 28: [96], 29: [97, 98, 99], 30: [100, 101, 102, 103], 31: [104, 105, 106], 32: [107, 108, 109, 110], 33: [111, 112], 34: [113, 114, 115, 116, 117, 118], 35: [119, 120, 121, 122, 123], 36: [124, 125, 126, 127, 128, 129], 37: [130, 131, 132], 38: [133, 134, 135, 136, 137], 39: [138, 139], 40: [140, 141, 142, 143, 144, 145], 41: [146, 147, 148, 149, 150], 42: [151, 152, 153, 154, 155], 43: [156, 157, 158, 159, 160, 161], 44: [162, 163, 164, 165, 166, 167], 45: [168, 169,

So the nice news is that the data is sorted by student id, so they are informally grouped together

In [24]:
demographics_raw.head(30)

,sid,school_year,male,race_ethnicity,birth_date,first_9th_school_year_reported,hs_diploma,hs_diploma_type,hs_diploma_date
0,1,2004,1,B,10869.0,2004.0,0,NaN,NaN
1,1,2005,1,H,10869.0,2004.0,0,NaN,NaN
2,1,2006,1,H,10869.0,2004.0,0,NaN,NaN
3,1,2007,1,H,10869.0,2004.0,0,NaN,NaN
4,2,2006,0,W,11948.0,NaN,1,Standard Diploma,05jun2008
5,2,2007,0,B,11948.0,NaN,1,College Prep Diploma,24may2009
6,3,2006,1,H,11724.0,2007.0,0,NaN,NaN
7,3,2006,1,B,11724.0,2007.0,0,NaN,NaN
8,3,2007,1,B,11724.0,2007.0,0,NaN,NaN
9,4,2005,0,B,12717.0,2007.0,1,Standard Diploma,04jun2009


Looking at the list of the first 30 entries, we see the first time things look weird in regard to gender is with student is 7

In [49]:
one_student = demographics_raw[demographics_raw['sid'] == 7]
one_student

,sid,school_year,male,race_ethnicity,birth_date,first_9th_school_year_reported,hs_diploma,hs_diploma_type,hs_diploma_date
16,7,2004,1,H,10960.0,2005.0,1,Standard Diploma,14may2008
17,7,2005,1,H,10960.0,2005.0,1,Standard Diploma,14may2008
18,7,2006,1,H,10960.0,2005.0,1,Standard Diploma,14may2008
19,7,2007,0,H,10960.0,2005.0,1,Standard Diploma,14may2008
20,7,2008,1,H,10960.0,2005.0,1,Standard Diploma,14may2008


So in a scenario like this, we need to figure out how to address it...in this case it looks like the female mark may have been a misentry. So how would we find this --> use the mode! Find the mode for each student's male value, if the row['male'] does not equal the mode, update it to the mode. 

In [57]:
mode = one_student['male'].mode().to_numpy()

In [58]:
mode

array([1])

In [62]:
one_student['male'][:] = mode[0]
one_student

/Users/kiara/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sid,school_year,male,race_ethnicity,birth_date,first_9th_school_year_reported,hs_diploma,hs_diploma_type,hs_diploma_date
16,7,2004,1,H,10960.0,2005.0,1,Standard Diploma,14may2008
17,7,2005,1,H,10960.0,2005.0,1,Standard Diploma,14may2008
18,7,2006,1,H,10960.0,2005.0,1,Standard Diploma,14may2008
19,7,2007,1,H,10960.0,2005.0,1,Standard Diploma,14may2008
20,7,2008,1,H,10960.0,2005.0,1,Standard Diploma,14may2008


In [38]:
#get a list of modes by student id
male_modes = demographics_raw.groupby('sid', as_index=False)['male']

In [39]:
male_modes.head()

0        1
1        1
2        1
3        1
4        0
        ..
87529    0
87530    0
87531    0
87532    0
87533    0
Name: male, Length: 82963, dtype: int64

In [56]:
demographics_raw.head(100)

,sid,school_year,male,race_ethnicity,birth_date,first_9th_school_year_reported,hs_diploma,hs_diploma_type,hs_diploma_date
0,1,2004,1,B,10869.0,2004.0,0,NaN,NaN
1,1,2005,1,H,10869.0,2004.0,0,NaN,NaN
2,1,2006,1,H,10869.0,2004.0,0,NaN,NaN
3,1,2007,1,H,10869.0,2004.0,0,NaN,NaN
4,2,2006,0,W,11948.0,NaN,1,Standard Diploma,05jun2008
...,...,...,...,...,...,...,...,...,...
95,27,2009,1,H,11701.0,2007.0,0,NaN,NaN
96,28,2009,1,B,11702.0,NaN,0,NaN,NaN
97,29,2004,1,W,10972.0,2005.0,0,NaN,NaN
98,29,2005,1,W,10972.0,2005.0,0,NaN,NaN


In [ ]:
#the question is are there any where the mode